## Introduction
This project performs sentiment analysis on IMDB movie reviews using various deep learning models, including BERT and DistilBERT. The goal is to classify reviews as positive or negative.

## Install and Import Libraries
We install and import the necessary libraries such as Transformers, Torch, Datasets, scikit-learn, nltk and Matplotlib.


In [1]:
!pip install transformers torch datasets scikit-learn nltk
!pip install --upgrade datasets fsspec
from transformers import AutoTokenizer , AutoModelForSequenceClassification
from datasets import load_dataset
import matplotlib.pyplot as plt
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


## Load and Explore the Dataset
We use the publicly available IMDB dataset from the `datasets` library.

In [3]:
dataset=load_dataset("imdb")
print(dataset)

for i in range(5):
    print("\nReview:", dataset['train'][i]['text'])
    print("Label:", dataset['train'][i]['label'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
print(dataset)

for i in range(5):
    print("\nReview:", dataset['train'][i]['text'])
    print("Label:", dataset['train'][i]['label'])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Review: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and 

## Data Preprocessing
We use a tokenizer to convert text into numerical tokens that the model can understand.

We use `max_length=128` to ensure uniform input size for batching. The tokenizer outputs `input_ids` and `attention_mask`.

In [5]:
# Chooce each model we will use (Here we are gonna use bert)
model_checkpoint = "bert-base-uncased"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

## Create DataLoader
We prepare batches of data for training.

Batching improves computational efficiency and allows parallel processing.

In [9]:
from torch.utils.data import DataLoader

batch_size = 16

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

## Check the shape of first batch
for batch in train_dataloader:
    print(batch)
    break

In [10]:
for batch in train_dataloader:
    print(batch)
    break

{'label': tensor([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]), 'input_ids': tensor([[  101,  1045,  2514,  ..., 11088,  1012,   102],
        [  101,  2053, 27594,  ...,  1055,  2439,   102],
        [  101,  1999,  1996,  ...,  1037,  2402,   102],
        ...,
        [  101,  2770,  2041,  ...,  7987,  1013,   102],
        [  101,  1045,  4669,  ...,  1011,  2009,   102],
        [  101,  5409,  3185,  ...,  1013,  1028,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


## Define the Model
We use a pre-trained BERT model for sequence classification with 2 output labels (positive, negative).

Also, We fine-tune a pre-trained BERT model to adapt it to our classification task.


In [11]:
from transformers import BertForSequenceClassification

model_checkpoint = "bert-base-uncased"

model = BertForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Training the Model
We train the model over multiple epochs, updating weights to minimize loss.

We use AdamW optimizer and CrossEntropyLoss for binary classification. Training runs for 2 epochs.


In [12]:
from torch import nn
from torch.optim import AdamW

loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
epochs = 2
losses_per_epoch= []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in train_dataloader:
        # خد الداتا من الـ Batch
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)

        # 1. Zero Gradients
        optimizer.zero_grad()

        # 2. Forward Pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        # 3. Backward Pass
        loss.backward()

        # 4. Update Weights
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Average Loss: {avg_loss:.4f}")
    losses_per_epoch.append(avg_loss)


Epoch 1/2


## Evaluate the Model
We check accuracy on the test dataset.

We use no_grad to avoid updating weights and reduce memory usage during evaluation.

We visualize true vs predicted labels by using Confusion Matrix .

In [ ]:
## Test the model and calculate the accuracy and Confusion Matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
model.eval()
correct = 0
total = 0

all_labels = []
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix For Bert')
plt.show()



Test Accuracy: 0.8843


## Save the Model
We save our trained model for future use.

In [ ]:
torch.save(model.state_dict(), "bert_sentiment_model.pth")

In [ ]:
# from google.colab import files
# files.download("bert_sentiment_model.pth")

## Load the Model
We can reload the model without retraining.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
## Use this code after uploading model to make some test
from transformers import BertForSequenceClassification
import torch
model_checkpoint = "bert-base-uncased"

model = BertForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("bert_sentiment_model.pth", map_location=device))
model.eval()

## Test the model
We can make some tests by applying different sentences

In [ ]:
# make some tests
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        label = "Positive 😊" if prediction == 1 else "Negative 😞"
    return label
print(predict_sentiment("I love this movie. It's amazing!"))
print(predict_sentiment("This was terrible"))

In [ ]:
## Check by writing any sentence you want
while True:
    user_input = input("Write your sentence you want to predict or write quit to close: ")
    if user_input.lower() == 'quit':
        break
    result = predict_sentiment(user_input)
    print(f"Predicted Sentiment: {result}")

## Define the Second Model (Distibert)


In [ ]:
## Start with second model (Distibert)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import  torch
rom torch import nn
from torch.optim import AdamW

model_checkpoint = "distilbert-base-uncased"

# Downlaoding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Downloadind model
distilbert_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# نقل الموديل للـ device
distilbert_model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(distilbert_model.parameters(), lr=2e-5)

## Training the other Model
We train the model over multiple epochs, updating weights to minimize loss.

We use AdamW optimizer and CrossEntropyLoss for binary classification. Training runs for 2 epochs.


In [ ]:
## Start Staring new model
epochs = 2
losses_per_epoch_distilber = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    distilbert_model.train()

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 1. Zero Gradients
        optimizer.zero_grad()

        # 2. Forward Pass
        outputs = distilbert_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()

        # 3. Backward Pass
        loss.backward()

        # 4. Update Weights
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Average Loss: {avg_loss:.4f}")
    losses_per_epoch_distilber.append(avg_loss)

## Evaluate the new Model
We check accuracy on the test dataset.

We use no_grad to avoid updating weights and reduce memory usage during evaluation.

We visualize true vs predicted labels by using Confusion Matrix .

In [ ]:
## Test the model and calculate the accuracy and Confusion Matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

distilbert_model.eval()

correct = 0
total = 0
all_labels = []
all_predictions = []


with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = distilbert_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Distilbert')
plt.show()


## Save the Model
We save our trained model for future use.

In [ ]:
## Save new model and downloaded it
torch.save(model.state_dict(), "distilbert_sentiment_model.pth")
from google.colab import files
files.download("distilbert_sentiment_model.pth")

In [ ]:
## Use this code after uploading model to make some test
from transformers import BertForSequenceClassification
import torch
model_checkpoint = "distilbert-base-uncased"

distilbert_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distilbert_model.load_state_dict(torch.load("bert_sentiment_model.pth", map_location = device))
distilbert_model.eval()

## Visualize Training Loss for both model
We plot the loss per epoch to check convergence.

In [ ]:
## Plot Training loss per epoch
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(losses_per_epoch, marker='o', label='BERT')
plt.plot(losses_per_epoch_distilber, marker='s', label='DistilBERT')

plt.title('Training Loss per Epoch for BERT vs. DistilBERT')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.grid()
plt.show()

## Build streamlit


In [ ]:
## Build simple Stramlit
%%writefile sentiment_app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# uploadind Model and tokenizer
@st.cache_resource
def load_model():
    model_checkpoint = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=2
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.load_state_dict(torch.load("bert_sentiment_model.pth", map_location=device))
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

# عنوان
st.title("Sentiment Analysis App ❤️")
st.write("Write anything to check if it is Positive or Negative.")

# إدخال المستخدم
user_input = st.text_area("write your sentence here:")

if st.button("Expect"):
    if user_input.strip() == "":
        st.warning("Please write your sentence first!")
    else:
        inputs = tokenizer(
            user_input,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=128
        )

        with torch.no_grad():
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1).item()

        label = "Positive 😊" if prediction == 1 else "Negative 😞"
        st.success(f"Expect: {label}")
